In [59]:
import ast

import pandas as pd
import numpy as np
import os
import ast

In [60]:
files_path = r'../../temp/test_dataset.xlsx'
# df = pd.read_excel('../../temp/test_dataset.xlsx')

In [61]:
# df['content'] = df['content'].apply(ast.literal_eval)

In [62]:
# df['content'] = df['content'].apply(lambda x: ''.join(x))
# df['content']

In [63]:
# df = df[['content']]
# df.to_excel('../../temp/pre_test_dataset.xlsx', index=False)

In [64]:
class NewsAnalysis:
    def __init__(self, files_path):
        self.files_path = files_path
        self.data = None
        
        self.max_relations_per_row  = 10

    def read_data(self):
        # 读取Excel文件
        self.original_data = pd.read_excel(self.files_path, sheet_name='Sheet2')
        self.data = self.original_data.copy()

    def transform_content(self):
        # 转换content列中的字符串表示的list为实际的list，然后合并为字符串
        self.data['content'] = self.data['content'].apply(ast.literal_eval)
        self.data['content'] = self.data['content'].apply(lambda x: ','.join(x))
        
    def find_closest_index(self, content, position_names, target_name):
        # 找到最接近的职位索引
        min_distance = float('inf')
        best_match = (-1, -1)  # 存储最佳匹配的起始和结束索引
        for position in position_names:
            if position in content:
                pos_index = content.index(position)
                name_index = content.index(target_name)
                distance = abs(pos_index - name_index)
                if distance < min_distance:
                    min_distance = distance
                    start_idx = pos_index
                    end_idx = pos_index + len(position) - 1
                    best_match = (start_idx, end_idx)
        if best_match == (-1, -1):
            print(f"未找到匹配的职位'{position_names}'接近'{target_name}'")
        
        return best_match

    def analyze_positions(self):
        # 初始化存储分析结果的列表
        max_relations_per_row = self.max_relations_per_row
        for i in range(1, max_relations_per_row + 1):
            self.data[f'实体关系{i}'] = None  # 添加足够的列来存储关系
        
        # 初始化存储分析结果的列表
        results = []
        previous_content = None  # 用于存储上一行的内容
        
        # 遍历每一行
        for index, row in self.data.iterrows():
            content = row['content']
            if pd.isna(content) and previous_content is not None:
                # 如果当前行的content为空，则使用上一行的content
                content = previous_content
                print(f"Warning: Row {index} has empty content, use previous content")
            elif pd.isna(content):
                # 如果连续行的content为空，则跳过
                continue
                
            entities = [
                ('来访城市', row.get('来访城市'), '来访人姓名', row.get('来访人姓名'), '来自'),
                ('来访人职位', row.get('来访人职位'), '来访人姓名', row.get('来访人姓名'), '担任'),
                ('受访城市', row.get('受访城市'), '接访人姓名', row.get('接访人姓名'), '来自'),
                ('接访人职位', row.get('接访人职位'), '接访人姓名', row.get('接访人姓名'), '担任'),
                ('来访人姓名', row.get('来访人姓名'), '接访人姓名', row.get('接访人姓名'), '接待')
            ]

            relation_set = set()
            # 遍历实体对
            for entity1_type, entity1_value, entity2_type, entity2_value, relation in entities:
                if pd.notna(entity1_value) and pd.notna(entity2_value) and entity1_value in content and entity2_value in content:
                    start_idx1 = content.index(entity1_value)
                    end_idx1 = start_idx1 + len(entity1_value) - 1  
                    start_idx2 = content.index(entity2_value) - 1
                    end_idx2 = start_idx2 + len(entity2_value)
                    results.append(f"{{[{start_idx1},{end_idx1}],{entity1_value}}}, {{[{start_idx2},{end_idx2}],{entity2_value}}},{relation}")

                    # 特殊处理职位，去除城市名称后重新寻找
                    if '职位' in entity1_type:
                        possible_positions = [entity1_value, entity1_value.replace(row.get('受访城市', '') + '市', '').strip()]
                        start_idx1, end_idx1 = self.find_closest_index(content, possible_positions, entity2_value)

                    relation_desc = f"{{[{start_idx1},{end_idx1}], {entity1_value}}}, {{[{start_idx2},{end_idx2}], {entity2_value}}}, {relation}"
                    if relation_desc not in relation_set:
                        relation_set.add(relation_desc)
                        
                else:
                    print(f"在content中未找到 '{entity1_value}' 或 '{entity2_value}'。")
            
            # 存储关系到列
            for i, relation in enumerate(relation_set):
                if i < max_relations_per_row:
                    self.data.at[index, f'实体关系{i+1}'] = relation
                    
            previous_content = content  # 更新上一行的内容为当前行

        return results
    
    def save_results(self, save_path=r'../../temp'):
        self.data = self.data[['content'] + [f'实体关系{i+1}' for i in range(self.max_relations_per_row)]]
        self.data = self.data.rename(columns={'content': '文本内容'})
        
        # 保存分析结果
        self.data.to_excel(os.path.join(save_path, 'final_test_dataset.xlsx'), index=False)
        
        print(f"Analysis results saved to {os.path.join(save_path, 'final_test_dataset.xlsx')}")
        


In [65]:

# 示例用法
# files_path = '/path/to/your/excel.xlsx'
news_analysis = NewsAnalysis(files_path)
news_analysis.read_data()
news_analysis.transform_content()
results = news_analysis.analyze_positions()
for result in results:
    print(result)
news_analysis.save_results()

In [19]:
import paddle

In [20]:
paddle.utils.run_check()
paddle.set_device('gpu')